In [1]:
import requests
import ast
import datetime
import pandas as pd
import re
import math
from google.colab import files
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pathlib
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
import io
import shutil
import glob
import time
import warnings
from pytz import timezone
warnings.filterwarnings("ignore")

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [4]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

class GoogleAPIConnection:
  # def __init__(self):
  #   # upload pickle.token file for google drive API Authentication
  #   self.uploaded = files.upload()

  def intiate_gdAPI(self):
    try:
      # If modifying these scopes, delete the file token.pickle.
      SCOPES = ['https://www.googleapis.com/auth/drive']
      """Shows basic usage of the Drive v3 API.
        Prints the names and ids of the first 10 files the user has access to.
        """
      creds = None
      # The file token.json stores the user's access and refresh tokens, and is
      # created automatically when the authorization flow completes for the first
      # time.
      if os.path.exists('token.json'):
          creds = Credentials.from_authorized_user_file('token.json', SCOPES)
      # If there are no (valid) credentials available, let the user log in.
      if not creds or not creds.valid:
          if creds and creds.expired and creds.refresh_token:
              creds.refresh(Request())
          else:
              flow = InstalledAppFlow.from_client_secrets_file(
                  'credentials.json', SCOPES)
              creds = flow.run_local_server(port=0)
          # Save the credentials for the next run
          with open('token.json', 'w') as token:
              token.write(creds.to_json())

      service = build('drive', 'v3', credentials=creds)
      return service
    except Exception as e:
      print('Google Drive API connection Exception=>', e)
      return None

  
  def upload_file(self, service, filename, filepath, folder_id, mime_type):
    #print(filepath)
    try:
      file_metadata = {'name': filename,
                      'parents': [folder_id]}
      media = MediaFileUpload(filepath, mimetype=mime_type)
      file = service.files().create(body=file_metadata,
                                          media_body=media,
                                          fields='id').execute()
      print('File ID: %s upload' % file.get('id'))
      print('File upload success')
    except Exception as e:
      print('file upload exception=>', e)

  def search_file(self, service, file_name, mime_type, folder_id, search_in_folder=False):
    try:
      page_token = None
      c = False
      while True:
        if search_in_folder==False:
          if mime_type == 'text/plain':
            response = service.files().list(q="mimeType='text/plain'",
                                                  spaces='drive',
                                                  fields='nextPageToken, files(id, name)',
                                                  pageToken=page_token).execute()
          elif mime_type == 'application/json':
            response = service.files().list(q="mimeType='application/json'",
                                                  spaces='drive',
                                                  fields='nextPageToken, files(id, name)',
                                                  pageToken=page_token).execute() 
          elif mime_type == 'image/png' or mime_type == 'image/jpg':
            response = service.files().list(q="mimeType='text/plain'",
                                                  spaces='drive',
                                                  fields='nextPageToken, files(id, name)',
                                                  pageToken=page_token).execute()
          else:
            response = service.files().list(spaces='drive',
                                                  fields='nextPageToken, files(id, name)',
                                                  pageToken=page_token).execute()
        else:
          response = service.files().list(q="parents in '{}'".format(folder_id),
                                                spaces='drive',
                                                fields='nextPageToken, files(id, name)',
                                                pageToken=page_token).execute()
        for file in response.get('files', []):
            # Process change
            
            ff = file.get('name')
            if ff == file_name:
                c = True
                print('Found file: %s (%s)' % (file.get('name'), file.get('id')))
                break
        if c == True:
            break
        page_token = response.get('nextPageToken', None)
        if page_token is None:
          return 0
      return file.get('id')
    except Exception as e:
      print(e)
      return 0

  def download_files(self, service, file_path, file_id, check=True):
      file = pathlib.Path(file_path)
      if file.exists() and check==True:
          print("File exist")
      else:
          print("File not exist")
          # file_id = '1RZlYb9ufdqA5NjC5oJ8BgAPGW02riZTB'
          # file_name = 'yolov3.cfg'
          request = service.files().get_media(fileId=file_id)
          #request = service.files().export_media(fileId=file_id, mimeType='application/pdf')
          fh = io.BytesIO()
          downloader = MediaIoBaseDownload(fh, request)
          done = False
          while done is False:
              status, done = downloader.next_chunk()
              print("Download %d%%." % int(status.progress() * 100))
          fh.seek(0)
          with open(file_path,'wb') as f:
              shutil.copyfileobj(fh,f)

#Initialize GAPI Object
objGAPI = GoogleAPIConnection()


In [5]:
%ls

sample_data/  token.json


In [6]:
#Download all supporting files in helpers foler if doesn't exists
service = objGAPI.intiate_gdAPI()
# file_id = objGAPI.search_file(service, 'ck8sproject-e1fb187dcd6d.json', 'application/json', "folder_id")
objGAPI.download_files(service, 'Test.zip', '13TN3XcmrJuk-Z0J6COMXBhKv0DNeVsjH')

File not exist
Download 27%.
Download 55%.
Download 82%.
Download 100%.


In [7]:
!unzip Test.zip

Archive:  Test.zip
   creating: Test/
   creating: Test/ip/
  inflating: Test/process_video_subs.py  
  inflating: Test/video_audio_subs_model7.pb  


In [8]:
%cd Test/
%ls

/content/Test
ip/  process_video_subs.py  video_audio_subs_model7.pb


In [19]:
!python process_video_subs.py --video=ip/Volleyball.mp4 --output=events





2021-05-27 13:50:05.763122: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-05-27 13:50:05.763415: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557885e2c840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-27 13:50:05.763451: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-05-27 13:50:05.765428: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-27 13:50:05.906187: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-27 13:50:05.907102: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557885e2cf40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-

In [20]:
!python process_video_subs.py --video=ip/Volleyball.mp4 --output=violence_score





2021-05-27 13:58:43.017103: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-05-27 13:58:43.017365: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a9ff43a840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-27 13:58:43.017404: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-05-27 13:58:43.019361: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-27 13:58:43.150652: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-27 13:58:43.151589: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a9ff43af40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-